In [1]:
import numpy as np
import ufl
from mpi4py import MPI
from petsc4py import PETSc
from slepc4py import SLEPc
import pyvista as pv
from dolfinx import mesh, fem, plot

Create domain! A rectangle (1x2) with 10 elements in each direction

In [2]:
domain = mesh.create_rectangle(comm=MPI.COMM_WORLD, points=[(0, 0), (2, 2)], n=(100, 100))
domain

create function space

In [3]:
V = fem.functionspace(domain, ("Lagrange", 1))

In [4]:
# 2. Apply Dirichlet BC: u = 0 on boundary ∂Ω
def boundary(x):
    return np.isclose(x[0], 0.0) | np.isclose(x[0], 2.0) | np.isclose(x[1], 0.0) | np.isclose(x[1], 2.0)

In [5]:
facets = mesh.locate_entities_boundary(domain, domain.topology.dim - 1, boundary)
dofs = fem.locate_dofs_topological(V, domain.topology.dim - 1, facets)
bc = fem.dirichletbc(PETSc.ScalarType(0), dofs, V) # drichlet boundary conditions

In [6]:
# 3. Define bilinear and linear forms
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
a = fem.form(ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx)
m = fem.form(ufl.inner(u, v) * ufl.dx)

In [7]:
import dolfinx.fem.petsc
# 4. Assemble matrices A and M
A = fem.petsc.assemble_matrix(a, bcs=[bc])
A.assemble()
M = fem.petsc.assemble_matrix(m, bcs=[bc])
M.assemble()

In [8]:
# 5. Solve the generalized eigenvalue problem A u = λ M u
eigensolver = SLEPc.EPS().create()
eigensolver.setOperators(A, M)
# eigensolver.setProblemType(SLEPc.EPS.ProblemType.HEP)
eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)

tol = 1e-3
eigensolver.setTolerances(tol=tol)
eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL) #set 1 for maximum eigenvalues

# eigensolver.setType(SLEPc.EPS.Type.KRYLOVSCHUR)

# Get ST context from eps
# st = eigensolver.getST()

# Set shift-and-invert transformation
# st.setType(SLEPc.ST.Type.SINVERT)

eigensolver.setDimensions(nev=5)  # Get 10 eigenvalues
eigensolver.setFromOptions()
eigensolver.solve() 

n_conv = eigensolver.getConverged()
if MPI.COMM_WORLD.rank == 0:
    print(f"Number of converged eigenvalues: {n_conv}")


Number of converged eigenvalues: 7


In [ ]:
from dolfinx.fem import Function, functionspace
from dolfinx.mesh import CellType, compute_midpoints, create_unit_cube, create_unit_square, meshtags

try:
    import pyvista
except ModuleNotFoundError:
    print("pyvista is required for this demo")
    exit(0)

# If environment variable PYVISTA_OFF_SCREEN is set to true save a png
# otherwise create interactive plot
if pyvista.OFF_SCREEN:
    pyvista.start_xvfb(wait=0.1)

# Set some global options for all plots
transparent = False
figsize = 800

# 6. Extract and plot first few eigenfunctions
if MPI.COMM_WORLD.rank == 0:
    topology, cell_types, geometry = plot.vtk_mesh(V)
    points = domain.geometry.x
    for i in range(n_conv):
        eigval = eigensolver.getEigenvalue(i)
        error = eigensolver.computeError(i)
        print(f"Eigenvalue {i}: {eigval:.4f} with Error: {error}")

        r, _ = A.getVecs()
        eigensolver.getEigenvector(i, r)

        uh = fem.Function(V)
        uh.x.petsc_vec.setArray(r.array)
        uh.x.petsc_vec.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

        # Create PyVista grid
        values = uh.x.array.real
        grid = pv.UnstructuredGrid(topology, cell_types, points)
        grid.point_data["u"] = values
        grid.set_active_scalars("u")
        warped = grid.warp_by_scalar('u')
        
        
        # Define the number of contour levels
        contour_levels = 25

        # Generate contours using the scalar data
        contours = grid.contour(contour_levels, scalars="u")

        # A plotting window is created with two sub-plots, one of the scalar
        # values and the other of the mesh is warped by the scalar values in
        # z-direction
        subplotter = pv.Plotter(shape=(1, 2))
        subplotter.subplot(0, 0)
        subplotter.add_text("Scalar contour field", font_size=14, color="black", position="upper_edge")
        subplotter.add_mesh(grid, show_scalar_bar=True, cmap="jet")
        subplotter.add_mesh(contours, color="blue", line_width=2)
        subplotter.view_xy()

        contours_warp = warped.contour(contour_levels, scalars="u")
        
        subplotter.subplot(0, 1)
        subplotter.add_text("Warped function", position="upper_edge", font_size=14, color="black")
        
        subplotter.add_mesh(warped, show_scalar_bar=True, cmap="jet")
        subplotter.add_mesh(contours_warp, color="blue", line_width=2)


        # if pyvista.OFF_SCREEN:
        #     subplotter.screenshot(
        #         "2D_function_warp.png",
        #         transparent_background=transparent,
        #         window_size=[figsize, figsize],
        #     )
        # else:
        # subplotter.link_views()
        subplotter.show()
        


Eigenvalue 0: 1.0000 with Error: 0.44388874278307017


Widget(value='<iframe src="http://localhost:45119/index.html?ui=P_0x71ea4ff03250_14&reconnect=auto" class="pyv…

Eigenvalue 1: 4.9360 with Error: 0.0017080064234876082


Widget(value='<iframe src="http://localhost:45119/index.html?ui=P_0x71ea4ff02350_15&reconnect=auto" class="pyv…

Eigenvalue 2: 12.3425 with Error: 0.0005590393877088185


...

Eigenvalue 5: 24.6980 with Error: 2.659151454178337e-07


Widget(value='<iframe src="http://localhost:45119/index.html?ui=P_0x71ea4ff03610_19&reconnect=auto" class="pyv…

Eigenvalue 6: 32.1370 with Error: 0.00014021525968865904


Widget(value='<iframe src="http://localhost:45119/index.html?ui=P_0x71ea4ff03750_20&reconnect=auto" class="pyv…

: 